In [36]:
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english') + [word.strip("\n") for word in open("remove_words.txt", "r")])

In [25]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 2087-A02D

 Directory of C:\Users\Administrator\Desktop\Document-Similarity

11/30/2022  02:57 PM    <DIR>          .
11/10/2022  02:45 PM    <DIR>          ..
11/30/2022  02:18 PM    <DIR>          .ipynb_checkpoints
11/16/2022  03:10 PM           391,697 all_articles.csv
11/21/2022  10:50 AM    <DIR>          archive
11/22/2022  10:49 AM            19,471 base_matrix.png
11/29/2022  11:25 AM             5,967 bert_similarity.py
11/30/2022  02:57 PM            29,136 Classifier.ipynb
11/30/2022  11:44 AM             1,610 cosine_similarity_test.py
11/16/2022  04:19 PM    <DIR>          doc2vec
11/22/2022  10:00 AM             7,241 doc2vec_similarity.py
11/22/2022  10:36 AM           435,172 doc2vec_tokenized_articles.csv
11/23/2022  10:30 AM             4,124 evaluate_model.py
11/21/2022  10:50 AM    <DIR>          experiments
11/29/2022  02:38 PM             8,458 extract_bert_embeddings.py
11/30/2022  12:17 PM            10,

In [26]:
articles_df = pd.read_csv('all_articles.csv', usecols = ['article_title','text','relevant_campbell','relevant_kristen','mismatch'])

In [27]:
articles_df

,article_title,text,relevant_campbell,relevant_kristen,mismatch
0,Intervention was the right thing to do,News Intervention was the right thing to do FI...,0,0,0
1,SIX STATES FILE A CHALLENGE TO THE BIDEN ADMIN...,SIX STATES FILE A CHALLENGE TO THE BIDEN ADMIN...,0,0,0
2,Senator Reverend Warnock Introduces Bill to En...,Senator Reverend Warnock Introduces Bill to En...,1,0,1
3,10 Markets Where Sellers Are Cutting Home Pric...,Daily 10 Markets Where Sellers Are Cutting Hom...,1,1,0
4,"Biden touts Inflation Reduction Act, critics s...","Biden touts Inflation Reduction Act, critics s...",0,1,1
...,...,...,...,...,...
122,Jobs & Money - Women stand to get a better deal.,Jobs & Money - Women stand to get a better dea...,1,1,0
123,Pension off AVC rules; The Last Word,Pension off AVC rules; The Last Word THE G...,0,1,1
124,"Gold Stocks Favoured As Telecom, Bank Shares Fall",Business; Money Market And Share Market Gold S...,0,0,0
125,Wall of silence over 'flexible' Abbey fund,Wall of silence over 'flexible' Abbey fund ...,0,1,1


In [28]:
relevant_df = pd.read_csv("relevant_test.csv", usecols = ['word','frequency','frequency_scaled','relevancy_score','weighted_score'])
relevant_df

,word,frequency,frequency_scaled,relevancy_score,weighted_score
0,retirement,131,0.828025,8.557992,7.086235
1,mortgage,112,0.707006,8.136207,5.752350
2,pay,90,0.566879,10.098199,5.724457
3,income,69,0.433121,9.648579,4.179002
4,interest,59,0.369427,9.810026,3.624086
5,home,65,0.407643,8.745063,3.564867
6,worker,61,0.382166,9.214806,3.521582
7,credit,57,0.356688,9.350683,3.335275
8,need,56,0.350318,8.943111,3.132937
9,cost,50,0.312102,9.946045,3.104180


In [29]:
irrelevant_df = pd.read_csv("irrelevant_test.csv",  usecols = ['word','frequency','frequency_scaled','relevancy_score','weighted_score'])
irrelevant_df

,word,frequency,frequency_scaled,relevancy_score,weighted_score
0,fund,94,0.978947,10.782338,10.555342
1,million,75,0.778947,8.913129,6.942859
2,market,70,0.726316,9.293736,6.750187
3,price,62,0.642105,10.332009,6.634238
4,deal,60,0.621053,10.136530,6.295318
5,company,53,0.547368,10.641443,5.824790
6,bank,58,0.600000,9.325776,5.595466
7,bond,52,0.536842,10.136220,5.441549
8,customer,48,0.494737,8.968694,4.437143
9,sale,38,0.389474,10.708981,4.170866


In [57]:
articles_df['predicted'] = np.ones(len(articles_df))
all_articles_relevancy = {}
for index, row in articles_df.iterrows():
    
    #if index == 0:
    all_articles_relevancy[row['article_title']] = 0
    sentences = sent_tokenize(row['text'])

    for sentence in sentences:
        words_in_sentence = list(sentence.split(" "))
        for word_ in words_in_sentence:

            word_ = word_.lower()
            word_ = word_.strip()
            word_ = word_.replace(" ", "")
            word_ = word_.replace(",", "")
            word_ = word_.replace(".", "")
            word_ = word_.replace(":", "")
            word_ = word_.replace("/", "")
            word_ = word_.replace("-", "")
            word_ = word_.replace("(", "")
            word_ = word_.replace(")", "")

            if lemmatizer.lemmatize(word_) != 'ha' and lemmatizer.lemmatize(word_) != 'wa':
                word_ = lemmatizer.lemmatize(word_)

            if word_ not in stop_words and word_ not in string.punctuation:

                if word_ in list(relevant_df['word']):
                    #print("RELEVANT WORD")
                    #print(word_)
                    all_articles_relevancy[row['article_title']] += float(relevant_df[relevant_df['word'] == word_]['weighted_score'])
                    #print(all_articles_relevancy['article_title'])

                if word_ in list(irrelevant_df['word']):
                    #print("IRRELEVANT WORD")
                    #print(word_)
                    all_articles_relevancy[row['article_title']] -= float(irrelevant_df[irrelevant_df['word'] == word_]['weighted_score'])
                    #print(all_articles_relevancy['article_title'])



In [58]:
all_articles_relevancy

{'Intervention was the right thing to do': -222.16368918680848,
 "SIX STATES FILE A CHALLENGE TO THE BIDEN ADMINISTRATION'S STUDENT LOAN CANCELLATION PROGRAM": 14.78439795542315,
 'Senator Reverend Warnock Introduces Bill to Ensure Georgia Car Buyers, Automakers Fully Benefit from Cost-Cutting Tax Credits': 1.8489220677047964,
 '10 Markets Where Sellers Are Cutting Home Prices the Most': -119.33902674346534,
 'Biden touts Inflation Reduction Act, critics say student loan cancellation offsets gains': 1.5850149797426187,
 'ULTIMUTT DEAL Aldi’s new £40 Specialbuy item is perfect for dog owners': -26.726436047407347,
 "Students react to Biden's student debt relief plan": 30.366365226101166,
 "A GOP attorney general says people are 'celebrating prematurely' Biden's student-loan forgiveness, suggesting lawsuits are to come": -9.206033511538255,
 'Energy deal ban: Ofgem green lights major change for Brits - ‘More resilient and fair’': -163.40073085158926,
 'Couple who bought B&Q plant for jus

In [18]:
list(relevant_words['word'])

['retirement',
 'mortgage',
 'pay',
 'income',
 'interest',
 'home',
 'worker',
 'credit',
 'need',
 'cost',
 'health',
 'help',
 'many',
 'month']

In [53]:
irrelevant_df[irrelevant_df['word'] == 'bank']['weighted_score']

6    5.595466
Name: weighted_score, dtype: float64

In [44]:
word_

'management'

In [62]:
for key in all_articles_relevancy.keys():
    if all_articles_relevancy[key] < 0:
        articles_df.loc[articles_df['article_title'] == key, ['predicted']] = 0

In [70]:
pd.set_option('display.max_rows', 1000)

In [72]:
articles_df[['relevant_campbell', 'predicted']]

,relevant_campbell,predicted
0,0,0.0
1,0,1.0
2,1,1.0
3,1,0.0
4,0,1.0
5,0,0.0
6,1,1.0
7,0,0.0
8,0,0.0
9,0,0.0


In [65]:
num_ers = 0
for index, row in articles_df.iterrows():
    if row['relevant_campbell'] != row['predicted']:
        num_ers += 1

In [66]:
num_ers

28

0.2204724409448819